<a href="https://colab.research.google.com/github/pkwon35/Smart_Factory_APS_System/blob/main/%EA%B8%B0%EC%83%81%EC%B2%AD_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import requests
import numpy as np

# 공공데이터 API => 기상청 과거 날씨데이터 요청

In [ ]:
## 대표코드
## startdt , enddt 설정 (임의로했음)
## stnlds(지역) 대전(133)으로 임의로 넣어줌


import json

url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
my_key = '본인 키' # str 형식

params ={'serviceKey' : my_key, 
         'pageNo' : '1', 
         'numOfRows' : '999', # 1000 이상일시 응답에러
         'dataType' : 'JSON', 
         'dataCd' : 'ASOS', 
         'dateCd' : 'DAY', 
         'startDt' : '20170101',
         'endDt' : '20171231', 
         'stnIds' : '133' }


response = requests.get(url, params=params)
r_dict = json.loads(response.content)
r_response = r_dict.get("response")
r_body = r_response.get("body")
r_items = r_body.get("items")
r_item = r_items.get("item")

city = [] #도시
time = [] #일자
tem = [] #온도
rain = [] #강수여부
hum = [] #습도
snow = [] # 합계3시간 신적설 ( 3시간동안 내린 양 )


empty_data = pd.DataFrame()

for i in range(len(r_item)):
    time.append(r_item[i]['tm'])
    city.append(r_item[i]['stnNm'])
    tem.append(r_item[i]['avgTa'])
    rain.append(r_item[i]['sumRn'])
    hum.append(r_item[i]['avgRhm'])
    snow.append(r_item[i]['sumDpthFhsc'])

time = pd.Series(time)
city = pd.Series(city)
tem = pd.Series(tem)
rain = pd.Series(rain)
hum = pd.Series(hum)
snow = pd.Series(snow)

data = [ time, city, tem, rain, hum, snow ]

df = pd.concat(data, axis=1)

df.columns = ['일자','시도','기온','강수량','습도','신적설량']
df.replace('', 0,inplace=True)

df['일자'] = pd.to_datetime(df['일자'])
df['기온'] = round(df['기온'].astype('float32'),1)
df['강수량'] = round(df['강수량'].astype('float32'),1)
df['습도'] = round(df['습도'].astype('float32'),1)
df['신적설량'] = round(df['신적설량'].astype('float32'),1)

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      365 non-null    datetime64[ns]
 1   시도      365 non-null    object        
 2   기온      365 non-null    float32       
 3   강수량     365 non-null    float32       
 4   습도      365 non-null    float32       
 5   신적설량    365 non-null    float32       
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 11.5+ KB


,일자,시도,기온,강수량,습도,신적설량
0,2017-01-01,대전,3.5,0.0,81.500000,0.0
1,2017-01-02,대전,4.7,0.0,83.000000,0.0
2,2017-01-03,대전,3.3,0.0,77.099998,0.0
3,2017-01-04,대전,4.2,0.0,70.500000,0.0
4,2017-01-05,대전,4.3,0.0,71.300003,0.0
...,...,...,...,...,...,...
360,2017-12-27,대전,-5.1,0.0,47.400002,0.0
361,2017-12-28,대전,-1.6,0.0,57.900002,0.0
362,2017-12-29,대전,2.9,0.0,78.900002,0.0
363,2017-12-30,대전,3.5,0.9,80.800003,0.0


In [ ]:
temp01 = df.copy()

In [ ]:
date18_21 = pd.concat([temp01, df], axis=0)
date18_21.to_pickle('/content/drive/MyDrive/5조_스마트팩토리/진유훈/temp18_21.pkl')

In [ ]:
# df.set_index('일자').resample('w').max()

In [ ]:
data = pd.concat([df, adf])

# 주단위 리샘플해서 return

 - loc_weather( 시작일, 마지막일, 지역) <- str 형식으로 넣어줘야함
 - 999일을 초과할수없음 약 2년 몇개월~ ㅋㅋ
 - 지역별 주단위로 리샘플해서 나옴

In [ ]:
# ## 수도권
# 서울 = ['서울','강서','송파','남서울','동서울','신림','풍납','성수']  ### 기상CODE 108 / (서울) 수도권기상청
# 경기도북부 = ['동두천','남양주','파주','덕소','양주','일산','의정부','김포','진접','포천','구리','고양','화도']  ### 기상CODE 98 / (동두천) 수도권기상청
# 경기도남부 = ['하남','용인','광주','오산','수원','평택','이천','동탄','안성','양평','화성','여주','안산','광명','성남','비봉','의왕','미사리']   ### 기상CODE 119 / (수원) 수도권기상청
# 인천 = ['인천','가좌','부천','송도','서인천','대산']  ### 기상CODE 112 / (인천) 수도권기상청

# ## 경상도 
# 부산광역시 = ['부산','정관','회동동','서부산']       ### 기상CODE / 159 (부산) 부산지방기상청
# 울산광역시 = ['울산','언양','온산','남부']  ### 기상CODE / 152 (울산) 울산기상대	
# 대구광역시 = ['대구','하빈']                ### 기상CODE / 143 (대구) 대구지방기상청
# 경상북도 = ['포항','경주','칠곡','울진','문경','안동','약목','경산','단촌','고령','영덕','예천','구미','영양','성주','남포항' ]  ### 기상CODE 278 / (의성) 대구지방기상청
# 경상남도 =['함안','진해','김해','의령','양산','합천','칠서','진주','산청','함양','창녕','창원','밀양','하동','진영','사천','마산']  ### 기상CODE 263 / (의령군) 창원기상대


# ## 전라도
# 전라북도 =['정읍','이서','익산','완주','군산','남원','부안','장수','임실','전주','오수','군위','순창' ]  ### 기상CODE 146 / (전주) 전주기상지청
# 전라남도 =['순천','담양','여수','나주','영광','화순','장흥','해남','광양','노화도','목포','장성','무안','구례','군산한전주']   ### 기상CODE 156 / (광주) 광주지방기상청


# ## 충청도
# 대전광역시 = ['대전','장동']  ### 기상CODE 133 / (대전) 대전지방기상청
# 세종시 = ['세종','연기']  ### 기상CODE 239 / (세종) 대전지방기상청
# 충청북도 = [ '청주','음성','충주','옥천','청원','진천','보은','단양','옥산' ]  ### 기상CODE 131 / (청주) 청주기상지청	
# 충청남도 = [ '서산','당진','계룡','천안','아산','영동','청양','홍성','금산','태안','공주','예산','천북','부여','논산','보령' ]  ### 기상CODE 129 / (서산) 홍성기상대


# ## 강원도
# 강원도 = ['강릉','동해','원주','삼척','철원','횡성','옥계','고성','양양','평창','춘천','영월']  ### 기상CODE 114 / (원주) 강원지방기상청	

def loc_weather(start_date, end_date , location):
    import json
    import pandas as pd
    import requests
    import numpy as np


    def cat_location(x):
        if x == '서울': return 108
        elif x == '경기도북부': return 98
        elif x == '경기도남부': return 119
        elif x == '인천': return 112
        elif x == '부산광역시': return 159
        elif x == '대구광역시': return 152
        elif x == '울산광역시': return 143
        elif x == '경상북도': return 278
        elif x == '경상남도': return 263
        elif x == '전라북도': return 146
        elif x == '전라남도': return 156
        elif x == '대전광역시': return 133
        elif x == '세종시': return 239
        elif x == '충청북도': return 131
        elif x == '충청남도': return 129
        elif x == '강원도': return 114
        else: return 119 ## 해당 안될시 생판지역인 평택 기준 날씨로 변환

    location_code = cat_location(location)

    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    params ={'serviceKey' : 'ZKOx0KH7l+PcSZZNRvuI54pjFf5gbYeIa1ccvoUcbzlwPA7ZRd9AqYB+V6++N/urN+9OncLmDH9MvqvMu5SKbg==', 
            'pageNo' : '1', 
            'numOfRows' : '999', 
            'dataType' : 'JSON', 
            'dataCd' : 'ASOS', 
            'dateCd' : 'DAY', 
            'startDt' : start_date,
            'endDt' : end_date, 
            'stnIds' : location_code }


    response = requests.get(url, params=params)
    r_dict = json.loads(response.text)
    r_response = r_dict.get("response")
    r_body = r_response.get("body")
    r_items = r_body.get("items")
    r_item = r_items.get("item")

    # city = [] #도시
    time = [] #일자
    tem = [] #온도
    rain = [] #강수여부
    hum = [] #습도
    snow = [] # 합계3시간 신적설 ( 3시간동안 내린 양 )


    empty_data = pd.DataFrame()

    for i in range(len(r_item)):
        time.append(r_item[i]['tm'])
        # city.append(r_item[i]['stnNm'])
        tem.append(r_item[i]['avgTa'])
        rain.append(r_item[i]['sumRn'])
        hum.append(r_item[i]['avgRhm'])
        snow.append(r_item[i]['sumDpthFhsc'])

    time = pd.Series(time)
    # city = pd.Series(city)
    tem = pd.Series(tem)
    rain = pd.Series(rain)
    hum = pd.Series(hum)
    snow = pd.Series(snow)

    data = [ time, tem, rain, hum, snow ]

    df = pd.concat(data, axis=1)

    df.columns = ['일자','기온','강수량','습도','신적설량']
    df.replace('', 0,inplace=True)

    df['일자'] = pd.to_datetime(df['일자'])
    df['기온'] = round(df['기온'].astype('float32'),1)
    df['강수량'] = round(df['강수량'].astype('float32'),1)
    df['습도'] = round(df['습도'].astype('float32'),1)
    df['신적설량'] = round(df['신적설량'].astype('float32'),1)
    df['지역'] = location

    fin_df = df.set_index('일자').resample('w').mean()
    return fin_df



    

In [ ]:
### 유훈함수 breaking
import json
import pandas as pd
import requests
import numpy as np

# for location in ['서울','경기도북부','경기도남부','인천','부산광역시','대구광역시','울산광역시','경상북도','전라북도','전라남도','대전광역시','세종시','충청북도','충청남도','강원도']:
location = '경기도북부'
start_date='20200101'
end_date='20200331'

def cat_location(x):
    if x == '서울': return 108
    elif x == '경기도북부': return 98
    elif x == '경기도남부': return 119
    elif x == '인천': return 112
    elif x == '부산광역시': return 159
    elif x == '대구광역시': return 152
    elif x == '울산광역시': return 143
    elif x == '경상북도': return 278
    elif x == '경상남도': return 263
    elif x == '전라북도': return 146
    elif x == '전라남도': return 156
    elif x == '대전광역시': return 133
    elif x == '세종시': return 239
    elif x == '충청북도': return 131
    elif x == '충청남도': return 129
    elif x == '강원도': return 114
    else: return 119 ## 해당 안될시 생판지역인 평택 기준 날씨로 변환

location_code = cat_location(location)

url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

params ={'serviceKey' : 'ZKOx0KH7l+PcSZZNRvuI54pjFf5gbYeIa1ccvoUcbzlwPA7ZRd9AqYB+V6++N/urN+9OncLmDH9MvqvMu5SKbg==', 
        'pageNo' : '1', 
        'numOfRows' : '999', 
        'dataType' : 'JSON', 
        'dataCd' : 'ASOS', 
        'dateCd' : 'DAY', 
        'startDt' : start_date,
        'endDt' : end_date, 
        'stnIds' : location_code }


response = requests.get(url, params=params)
r_dict = json.loads(response.text)
r_response = r_dict.get("response")
r_body = r_response.get("body")
r_items = r_body.get("items")
r_item = r_items.get("item")

# city = [] #도시
time = [] #일자
tem = [] #온도
rain = [] #강수여부
hum = [] #습도
snow = [] # 합계3시간 신적설 ( 3시간동안 내린 양 )


empty_data = pd.DataFrame()

for i in range(len(r_item)):
    time.append(r_item[i]['tm'])
    # city.append(r_item[i]['stnNm'])
    tem.append(r_item[i]['avgTa'])
    rain.append(r_item[i]['sumRn'])
    hum.append(r_item[i]['avgRhm'])
    snow.append(r_item[i]['sumDpthFhsc'])

time = pd.Series(time)
# city = pd.Series(city)
tem = pd.Series(tem)
rain = pd.Series(rain)
hum = pd.Series(hum)
snow = pd.Series(snow)

data = [ time, tem, rain, hum, snow ]


df = pd.concat(data, axis=1)

df.columns = ['일자','기온','강수량','습도','신적설량']
df.replace('', 0,inplace=True)

df['일자'] = pd.to_datetime(df['일자'])
df['기온'] = round(df['기온'].astype('float32'),1)
df['강수량'] = round(df['강수량'].astype('float32'),1)
df['습도'] = round(df['습도'].astype('float32'),1)
df['신적설량'] = round(df['신적설량'].astype('float32'),1)


add_df = df.set_index('일자').resample('w').mean()
add_df['지역'] = location
add_df


    

,기온,강수량,습도,신적설량,지역
일자,,,,,
2020-01-05,-1.800000,0.020000,72.840004,0.0,경기도북부
2020-01-12,-0.028571,10.128572,84.728577,0.0,경기도북부
2020-01-19,-2.971429,0.028571,65.714287,0.0,경기도북부
2020-01-26,0.985714,0.000000,66.485718,0.0,경기도북부
2020-02-02,3.200000,0.028571,67.514290,0.0,경기도북부
2020-02-09,-4.000000,0.228571,58.700001,0.0,경기도북부
2020-02-16,4.800000,2.357143,77.371429,0.0,경기도북부
2020-02-23,-0.085714,1.485714,73.614288,0.0,경기도북부
2020-03-01,4.700000,3.971429,80.114288,0.0,경기도북부


In [ ]:
# locations = [서울,경기도북부,경기도남부,인천,부산광역시,대구광역시,울산광역시,경상북도,경상남도,전라북도,전라남도,대전광역시,세종시,충청북도,충청남도,강원도]

In [ ]:
# for loc in locations:
#     loc_weather('20200101', '20200331', str(loc))

loc_weather('20200101', '20200331', '경기도북부')

,기온,강수량,습도,신적설량
일자,,,,
2020-01-05,-1.800000,0.020000,72.840004,0.0
2020-01-12,-0.028571,10.128572,84.728577,0.0
2020-01-19,-2.971429,0.028571,65.714287,0.0
2020-01-26,0.985714,0.000000,66.485718,0.0
2020-02-02,3.200000,0.028571,67.514290,0.0
2020-02-09,-4.000000,0.228571,58.700001,0.0
2020-02-16,4.800000,2.357143,77.371429,0.0
2020-02-23,-0.085714,1.485714,73.614288,0.0
2020-03-01,4.700000,3.971429,80.114288,0.0


In [ ]:
서울

['서울', '강서', '송파', '남서울', '동서울', '신림', '풍납', '성수']

In [ ]:
# city = [] #도시
# time = [] #일자
# tem = [] #온도
# rain = [] #강수여부
# hum = [] #습도
# snow = [] # 합계3시간 신적설 ( 3시간동안 내린 양 )


# empty_data = pd.DataFrame()

# for i in range(len(r_item)):
#     time.append(r_item[i]['tm'])
#     city.append(r_item[i]['stnNm'])
#     tem.append(r_item[i]['avgTa'])
#     rain.append(r_item[i]['sumRn'])
#     hum.append(r_item[i]['avgRhm'])
#     snow.append(r_item[i]['sumDpthFhsc'])

# time = pd.Series(time)
# city = pd.Series(city)
# tem = pd.Series(tem)
# rain = pd.Series(rain)
# hum = pd.Series(hum)
# snow = pd.Series(snow)

# data = [ time, city, tem, rain, hum, snow ]

# df = pd.concat(data, axis=1)

# df.columns = ['일자','시도','기온','강수량','습도','신적설량']
# df.replace('', 0,inplace=True)

# df['일자'] = pd.to_datetime(df['일자'])
# df['기온'] = df['기온'].astype('float32')
# df['강수량'] = df['강수량'].astype('float32')
# df['습도'] = df['습도'].astype('float32')
# df['신적설량'] = df['신적설량'].astype('float32')

# df.info()
# df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      90 non-null     datetime64[ns]
 1   시도      90 non-null     object        
 2   기온      90 non-null     float32       
 3   강수량     90 non-null     float32       
 4   습도      90 non-null     float32       
 5   신적설량    90 non-null     float32       
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 2.9+ KB


,일자,시도,기온,강수량,습도,신적설량
0,2021-01-01,서울,-4.2,0.0,64.000000,0.0
1,2021-01-02,서울,-5.0,0.0,38.500000,0.0
2,2021-01-03,서울,-5.6,0.0,45.000000,0.0
3,2021-01-04,서울,-3.5,0.0,51.400002,0.0
4,2021-01-05,서울,-5.5,0.0,52.799999,0.0
...,...,...,...,...,...,...
85,2021-03-27,서울,12.1,25.5,70.900002,0.0
86,2021-03-28,서울,9.0,9.2,95.800003,0.0
87,2021-03-29,서울,10.2,0.2,62.299999,0.0
88,2021-03-30,서울,10.8,0.0,47.299999,0.0


# 향후 ''''타겟 기간'''' 에 강수량, 기온, 습도  
# 평년 날씨 

# 지방 별 지역코드..


In [1]:
# 90	속초	강원지방기상청	
# 212	홍천	춘천기상대
# 93	북춘천	춘천기상대	
# 216	태백	강원지방기상청
# 95	철원	강원지방기상청	
# 217	정선군	강원지방기상청
# 98	동두천	수도권기상청	
# 221	제천	청주기상지청
# 99	파주	수도권기상청	
# 226	보은	청주기상지청
# 100	대관령	강원지방기상청	
# 232	천안	대전지방기상청
# 101	춘천	춘천기상대	
# 235	보령	대전지방기상청
# 102	백령도	수도권기상청	
# 236	부여	대전지방기상청
# 104	북강릉	강원지방기상청	
# 238	금산	대전지방기상청
# 105	강릉	강원지방기상청	
# 239	세종	대전지방기상청
# 106	동해	강원지방기상청	
# 243	부안	전주기상지청
# 108	서울	수도권기상청	
# 244	임실	전주기상지청
# 112	인천	수도권기상청	
# 245	정읍	전주기상지청
# 114	원주	강원지방기상청	
# 247	남원	전주기상지청
# 115	울릉도	대구지방기상청	
# 248	장수	전주기상지청
# 119	수원	수도권기상청	
# 251	고창군	전주기상지청
# 121	영월	강원지방기상청	
# 252	영광군	광주지방기상청
# 127	충주	청주기상지청	
# 253	김해시	부산지방기상청
# 129	서산	홍성기상대	
# 254	순창군	전주기상지청
# 130	울진	안동기상대	
# 255	북창원	창원기상대
# 131	청주	청주기상지청	
# 257	양산시	울산기상대
# 133	대전	대전지방기상청	
# 258	보성군	광주지방기상청
# 135	추풍령	청주기상지청	
# 259	강진군	목포기상대
# 136	안동	안동기상대	
# 260	장흥	목포기상대
# 137	상주	대구지방기상청	
# 261	해남	목포기상대
# 138	포항	대구지방기상청	
# 262	고흥	광주지방기상청
# 140	군산	전주기상지청	
# 263	의령군	창원기상대
# 143	대구	대구지방기상청	
# 264	함양군	창원기상대
# 146	전주	전주기상지청	
# 266	광양시	광주지방기상청
# 152	울산	울산기상대	
# 268	진도군	목포기상대
# 155	창원	창원기상대	
# 271	봉화	대구지방기상청
# 156	광주	광주지방기상청
# 272	영주	안동기상대
# 159	부산	부산지방기상청	
# 273	문경	안동기상대
# 162	통영	부산지방기상청	
# 276	청송군	대구지방기상청
# 165	목포	목포기상대	
# 277	영덕	대구지방기상청
# 168	여수	광주지방기상청	
# 278	의성	대구지방기상청
# 169	흑산도	광주지방기상청	
# 279	구미	대구지방기상청
# 170	완도	목포기상대	
# 281	영천	대구지방기상청
# 172	고창	전주기상지청	
# 283	경주시	대구지방기상청
# 174	순천	광주지방기상청	
# 284	거창	울산기상대
# 177	홍성	홍성기상대	
# 285	합천	울산기상대
# 184	제주	제주지방기상청	
# 288	밀양	울산기상대
# 185	고산	제주지방기상청	
# 289	산청	창원기상대
# 188	성산	제주지방기상청	
# 294	거제	부산지방기상청
# 189	서귀포	제주지방기상청	
# 295	남해	부산지방기상청
# 192	진주	창원기상대			
# 201	강화	인천기상대			
# 202	양평	수도권기상청			
# 203	이천	수도권기상청			
# 211	인제	강원지방기상청			



# 경기는 어쩌누...